In [ ]:
import os
if not os.path.isdir("./download_google_drive/"):

In [ ]:
import matplotlib.pyplot as plt

# We define which sequences the functions is going to show and which view-angles
scondition = np.array(['nm-01','bg-01','cl-01'])
sviews = np.array(['000','018','036','054','072','090','108','126','144','162','180'])  

def plot_subject(subject, image, sviews, scondition,name):
  plt.figure(figsize=(14,4))
  count = 1
  for i, cond in enumerate(scondition):
    for j, view in enumerate(sviews):
      plt.subplot(len(scondition), len(sviews), count)
      path = data_base+slash+data_set_rep+slash+subject+slash+cond+slash+view+slash+image+'.png'
      plt.imshow(cv2.imread(path,0),cmap='gray')
      plt.xticks([])
      plt.yticks([])
      if(j==0):
        plt.ylabel(cond)
      if(i==len(scondition)-1):
        plt.xlabel(view)
      count +=1
  plt.subplots_adjust(wspace=0.05, hspace=0.01)
  plt.savefig(name+'.png')

In [ ]:
subject = '012'
plot_subject(subject,'1',sviews,scondition,'Original_GEI')

In [ ]:
import numpy as np
from os import listdir
import cv2
import os
from glob import glob
import time
t0 = time.clock()
import matplotlib.pyplot as plt

data_base = 'CASIA'
data_set = 'DatasetB'
data_set_rep = 'sil'
directorio = 'silhouettes'
condition = np.array(['bg-01','bg-02','cl-01','cl-02','nm-01','nm-02','nm-03','nm-04','nm-05','nm-06'])
formato = '.png'
visualize = 0
slash='/'

count = 1
## Iterate through each subject
for subject in range(1,63):
  print('\rSubject:',subject,end='')
  subject = str(subject).zfill(3)

## Iterate through the normal conditions (Target images)
  for dom_con in condition[4:]:
    view = '090'
    slash='/'
    rep_sil = data_base+slash+data_set_rep+slash+subject+slash+dom_con+slash+view+slash+'1.png'
    GEI_dom = cv2.imread(rep_sil,0)
    
## Iterate through each condition (Source images)
    for i in range(len(condition)):
  ## Iterate through view
      for vi in range(11):
        view = str(vi*18).zfill(3)
        slash='/'
        rep_sil = data_base+slash+data_set_rep+slash+subject+slash+condition[i]+slash+view+slash+'1.png'
        GEI = cv2.imread(rep_sil,0)
        
        # Save the GEIs
        inputdata = "CASIA/pix2pix/inputdata/"
        targetdata = "CASIA/pix2pix/targetdata/"
        cv2.imwrite(inputdata+'s'+str(subject).zfill(3)+'_'+str(count).zfill(5)+'.png',GEI)
        cv2.imwrite(targetdata+'s'+str(subject).zfill(3)+'_'+str(count).zfill(5)+'.png',GEI_dom)

        count +=1
        if(visualize):
          f = plt.figure()
          f.add_subplot(1,2, 1)
          plt.imshow(GEI, cmap='gray')
          f.add_subplot(1,2, 2)
          plt.imshow(GEI_target,cmap='gray')
          plt.show(block=True)  
        if cv2.waitKey(50) & 0xff==27:
          break

print('\tTime: {} s'.format(round((time.clock()-t0),2)))

In [ ]:
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
import matplotlib.pyplot as plt
import numpy as np

# Paths
PATH    = '/content/CASIA/pix2pix/'
inpath  = '/content/CASIA/pix2pix/inputdata'
outpath = '/content/CASIA/pix2pix/targetdata'

imgurls = !ls -1 "{inpath}"

np.random.seed(23)
np.random.shuffle(imgurls)

# imgurls = imgurls[:20000]
n = len(imgurls)
train_n = round(n*0.90)

# Lista random
randurls = np.copy(imgurls)

tr_urls = randurls[:train_n]
ts_urls = randurls[train_n:n]

print('Total images:',len(imgurls),' \nTraining:',len(tr_urls),' Testing:', len(ts_urls))

In [ ]:
IMG_WIDTH = 64
IMG_HEIGHT = 64


# Resize images
def resize(inimg, tgimg, height, width):
  inimg = tf.image.resize(inimg, [height, width])
  tgimg = tf.image.resize(tgimg, [height, width])
  return inimg, tgimg

# Normalize the pixel's values
def normalize(inimg, tgimg):
  inimg = (inimg/127) - 1
  tgimg = (tgimg/127) - 1
  return inimg, tgimg

@tf.function()
def random_jitter(inimg, tgimg):
  inimg, tgimg = resize(inimg, tgimg, int(IMG_WIDTH*1.02), int(IMG_HEIGHT*1.02))
  stacked_image = tf.stack([inimg, tgimg], axis = 0)
  cropped_image = tf.image.random_crop(stacked_image, size=[2,IMG_HEIGHT,IMG_WIDTH,1])
  inimg, tgimg = cropped_image[0], cropped_image[1]
  return inimg, tgimg

@tf.function()
def load_image(filename, augment = True):
  inimg = tf.image.rgb_to_grayscale(tf.image.decode_png(tf.io.read_file(inpath+'/'+filename),channels=3))
  tgimg = tf.image.rgb_to_grayscale(tf.image.decode_png(tf.io.read_file(outpath+'/'+filename),channels=3))
  
  inimg, tgimg = resize(inimg,tgimg, IMG_HEIGHT, IMG_WIDTH)
  
  if augment:
    inimg, tgimg = random_jitter(inimg, tgimg)
    
  inimg, tgimg = normalize(inimg, tgimg)
  return inimg, tgimg
  
def load_train_image(filename):
  return load_image(filename, True)

def load_test_image(filename):
  return load_image(filename, False)

In [ ]:
inimg, tgimg = load_train_image('s001_00001.png')
plt.imshow(inimg.numpy()[:,:,0]*+1/2,cmap='gray')
plt.show()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(tr_urls)
train_dataset = train_dataset.map(load_train_image,num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(200)

test_dataset = tf.data.Dataset.from_tensor_slices(ts_urls)
test_dataset = test_dataset.map(load_test_image,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(200)

for inimg, tgimg in train_dataset.take(5):
  f = plt.figure()
  f.add_subplot(1,2, 1)
  plt.title('Source image')
  plt.imshow((inimg[0,...,0]+1)/2,cmap='gray')
  f.add_subplot(1,2, 2)
  plt.title('Target image')
  plt.imshow((tgimg[0,...,0]+1)/2,cmap='gray')
  plt.show(block=True) 

In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *

def downsample(filters, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(Conv2D(filters, 
                    kernel_size = 4, 
                    strides=2, 
                    padding='same',
                    kernel_initializer=initializer, 
                    use_bias= not apply_batchnorm))

  if apply_batchnorm:
    result.add(BatchNormalization())

#   result.add(LeakyReLU())
  result.add(tf.keras.layers.LeakyReLU())

  return result

def upsample(filters, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)
  result = tf.keras.Sequential()
  result.add(Conv2DTranspose(filters, 
                             kernel_size = 4, 
                             strides=2, 
                             padding='same',
                             kernel_initializer=initializer, 
                             use_bias= False))
  result.add(BatchNormalization())
  if apply_dropout:
    result.add(Dropout(0.3))

  result.add(ReLU())
#   result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH,1])

  down_stack = [
    downsample(64, apply_batchnorm=False),                                      # (bs, 32, 32, 64)
    downsample(128),                                                            # (bs, 16, 16, 128)
    downsample(256),                                                            # (bs, 8,  4,  256)
    downsample(512),                                                            # (bs, 4,  4,  512)
  ]

  up_stack = [
    upsample(256, apply_dropout=True),                                          # (bs, 8,  8,  256)
    upsample(128, apply_dropout=False),                                         # (bs, 16, 16, 128)
    upsample(64, apply_dropout=False),                                          # (bs, 32, 32, 64)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(filters = 1, 
                                         kernel_size = 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')                     # (bs, 64, 64, 1)

  concat = tf.keras.layers.Concatenate()
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator = Generator()
gen_output = generator((inimg), training=False)
f = plt.figure()
f.add_subplot(1,2, 1)
plt.imshow((inimg[0,...,0]+1)/2,cmap='gray')
f.add_subplot(1,2, 2)
plt.imshow((gen_output[0,...,0]+1)/2,cmap='gray')
plt.show(block=True)
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
def Identificador_Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)
  inp = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, 1], name='input_image')
  tar = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, 1], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar])                                   # (bs, 64, 64, 2)

  down1 = downsample(64, False)(x)                                              # (bs, 32, 32, 64)
  down2 = downsample(128)(down1)                                                # (bs, 16, 16, 128)
  down3 = downsample(256)(down2)                                                # (bs, 8,  8,  256)
  last = tf.keras.layers.Conv2D(filters=1,
                                kernel_size = 4,
                                strides=1,
                                kernel_initializer=initializer,
                                padding = 'valid',
                                activation='sigmoid')(down3)                    # (bs, 5, 5, 1)
  

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
idiscriminator= Identificador_Discriminator()
disc_out = idiscriminator([(inimg), gen_output], training=False)
plt.figure(figsize=(4,4))
plt.imshow(disc_out[0,...,0], vmin=0, vmax=1, cmap='RdBu_r')
plt.colorbar()
plt.xticks([])
plt.yticks([])
plt.show()
disc_out.shape

tf.keras.utils.plot_model(idiscriminator, show_shapes=True, dpi=64)

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def idiscriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
  total_disc_loss = real_loss + generated_loss
  return total_disc_loss

LAMBDA = 400
def generator_loss(idisc_generated_output, gen_output, target):
  gan_loss1 = loss_object(tf.ones_like(idisc_generated_output), idisc_generated_output)
  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
  total_gen_loss = gan_loss1 + (LAMBDA * l1_loss)
  return total_gen_loss, gan_loss1, l1_loss

In [ ]:
import os
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
idiscriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
def generate_images(model, test_input, tar, save_filename = False, display_imgs = True):
  prediction = model(test_input, training=True)
  if save_filename:
    tf.keras.preprocessing.image.save_img(PATH+'outputs/'+save_filename+'.jpg', prediction[0,...])
  
  plt.figure(figsize=(7,7))

  display_list = [test_input[0,...,], tar[0,...,], prediction[0,...,]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i][...,0] * 0.5 + 0.5, cmap = 'gray')
    plt.axis('off')
  plt.show()

In [ ]:
import datetime
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

@tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as idisc_tape, tf.GradientTape() as rfdisc_tape:
    gen_output = generator(input_image, training=True)

    idisc_real_output      = idiscriminator([input_image, target], training=True)
    idisc_generated_output = idiscriminator([input_image, gen_output], training=True)

    gen_loss,ganloss1,l1loss = generator_loss(idisc_generated_output, gen_output, target)
    idisc_loss = idiscriminator_loss(idisc_real_output, idisc_generated_output)

  generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
  idiscriminator_gradients = idisc_tape.gradient(idisc_loss, idiscriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
  idiscriminator_optimizer.apply_gradients(zip(idiscriminator_gradients, idiscriminator.trainable_variables))
  
  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', ganloss1, step=epoch)
    tf.summary.scalar('gen_l1_loss', l1loss, step=epoch)
    tf.summary.scalar('idisc_loss', idisc_loss, step=epoch)

In [ ]:
from IPython.display import clear_output
import time

def fit(train_ds, epochs, test_ds):
  for epoch in range(epochs):
    start = time.time()

    imgi = 0
    for input_image, target in train_ds:
      print('\r','Epoch: '+str(epoch)+' - train: '+str(imgi)+'/'+str(len(tr_urls)),end='')
      imgi +=1
      train_step(input_image, target, epoch)

    imgi = 0
    for example_input, example_target in test_ds.take(3):
      generate_images(generator, example_input, example_target, str(imgi)+'_'+str(epoch), display_imgs=True)
      imgi +=1

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))

In [ ]:
EPOCHS = 10
fit(train_dataset, EPOCHS, test_dataset)

In [ ]:

idrange = [63,125]
def transf(name):
  img = tf.image.rgb_to_grayscale(tf.image.resize((tf.image.decode_png(tf.io.read_file(name),channels=3)),(64, 64)))
  img = img[np.newaxis]/127-1
  pred = np.array(255*((generator(img,training = False)[0]+1)/2))
  return pred

In [ ]:
import numpy as np
from numpy import ma
from os import listdir
import cv2
import os
from glob import glob
import time
t0 = time.clock()

data_base = 'CASIA'
data_set = 'DatasetB' 
data_set_rep = 'sil'
data_train = 'data'
directorio = 'silhouettes'	

train_   = np.array(['nm-01','nm-02','nm-03','nm-04'])
test_nm_ = np.array(['nm-05','nm-06'])
test_cl_ = np.array(['cl-01','cl-02'])
test_bg_ = np.array(['bg-01','bg-02'])
view = '090'	
formato = '.png'


data = {}
data["train_"] = train_
data["test_nm_"] = test_nm_
data["test_cl_"] = test_cl_
data["test_bg_"] = test_bg_

def transform_data(verbose=True):
  for subject in range(idrange[0],idrange[1]):
    subject = str(subject).zfill(3)
    if verbose:
      print('\r', 'Subject: ',subject, end='')
    for dset in data:
      condition = data[dset]
      for cond in condition:
        for view in range(11):
          view = str(view*18).zfill(3)
          GEI_path = data_base+slash+data_set_rep+slash+subject+slash+cond+slash+view+slash+'1.png'                 #CASIA/sil/104/nm-04/090/1.png
          GEI_save_path = data_base+slash+data_set_rep+slash+subject+slash+cond+slash+view+slash+file_name+'.png'   #CASIA/sil/104/nm-04/090/4.png
          GEI = transf(GEI_path)
          cv2.imwrite(GEI_save_path,GEI)
  if verbose:
    print("\tTransform_data: ",str(time.clock()-t0))

In [ ]:
import numpy as np
from numpy import ma
from os import listdir
import cv2
import os
from glob import glob
import time
t0 = time.clock()

data_base = 'CASIA'
data_set = 'DatasetB' 
data_set_rep = 'sil'
data_train = 'data'
directorio = 'silhouettes'	
views = np.array(['000','018','036','054','072','090','108','126','144','162','180'])
train_ = np.array(['nm-01','nm-02','nm-03','nm-04'])
test_nm_ = np.array(['nm-05','nm-06'])
test_cl_ = np.array(['cl-01','cl-02'])
test_bg_ = np.array(['bg-01','bg-02'])
formato = '.png'


data = {}
data["train_"] = train_
data["test_nm_"] = test_nm_
data["test_cl_"] = test_cl_
data["test_bg_"] = test_bg_


def split_flat(verbose=True):
  t0 = time.clock()
  for view in views:
    if verbose: 
      print('View: ', str(view))
  #   view = str(view*18).zfill(3)
    for dset in data:
      if verbose: 
        print(dset, ':',data[dset])
      condition = data[dset]
      matriz  = []
      etiqueta = []
      c1 =0
      for subject in range(idrange[0], idrange[1]):
        subject = str(subject).zfill(3)
  #       print('Subject: ', subject)
        for i in range(len(condition)):
          # Definimos el nombre de la imagen
          read_gei = data_base+slash+data_set_rep+slash+subject+slash+condition[i]+slash+view+slash+file_name+'.png'
          img = cv2.imread(read_gei,0)
  #         img[int(x1*0.12):int(x1*0.68),0:y1]=0

          try:
            matriz.append(img.flatten())
            etiqueta.append(int(subject))
          except:
            print('Error saving:', read_gei)

          if cv2.waitKey(1) & 0xff==27:
            break

      matriz = np.array(matriz)
      etiqueta = np.array(etiqueta)[np.newaxis].T
      directory_data = data_base+slash+data_train+slash+view+slash
      if not os.path.exists(directory_data):
          os.makedirs(directory_data)
      np.savetxt(directory_data+dset+'data.dat', matriz)
      np.savetxt(directory_data+dset+'target.dat', etiqueta)
      if verbose:
        print(dset,'data.dat-> ',matriz.shape)
        print(dset,'data.dat-> ',etiqueta.shape)

  cv2.destroyAllWindows()
  if verbose:
    print(str(time.clock()-t0))

In [ ]:
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import svm


def normalizacion(data):
	u = np.mean(data,axis=0)
	s = np.std(data,axis=0)
	data = (data - u)/s
	return (data,u,s)
def normalizacion2(data,u,s):
	data = (data - u) / s
	return data

datasets = ['test_nm_','test_bg_','test_cl_']
data_base = 'CASIA'
data_train = 'data'
slash = '/'

CCR_nm = np.zeros((11,11))
CCR_bg = np.zeros((11,11))
CCR_cl = np.zeros((11,11))

def test_model(iteration):



  for view_g,galery_view in enumerate(views):
    # Cargamos la base de datos
    carpeta = data_base+slash+data_train+slash+galery_view+slash

    trainX = np.loadtxt(carpeta+'train_data.dat')

    # Escalamos los datos
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX = scaler.transform(trainX)

    trainY = np.loadtxt(carpeta+'train_target.dat')

    _ ,componentes_original = trainX.shape

    # Aplicamos PCA
    pre = 0.9999
    pca = PCA(pre)
    pca.fit(trainX)
    componentes_PCA = pca.n_components_
    trainX = pca.transform(trainX)

    logisticRegr = LinearDiscriminantAnalysis(solver = 'lsqr',shrinkage=0.2)
    logisticRegr.fit(trainX, trainY)


    # Calculamos su score
    score = logisticRegr.score(trainX, trainY)
    print('Dataset: Train',' shape test: ',trainX.shape,' correcto: ',np.round(score,4), 'Angle:', galery_view)
    # Realizamos pruebas en los datasets de testeo
    for view_p,prove_view in enumerate(views):
      carpeta_p = data_base+slash+data_train+slash+prove_view+slash
      for i in range(len(datasets)):
        testX = np.loadtxt(carpeta_p+datasets[i]+'data.dat')
        testX = scaler.transform(testX)
        # testX = normalizacion2(testX,mean,st)
        testX = pca.transform(testX)
        testY = np.loadtxt(carpeta_p+datasets[i]+'target.dat')

        # Calculamos su score
        score = logisticRegr.score(testX, testY)
        certeza = logisticRegr.predict(testX)

        # Mostramos la matriz de confusion
        # print(confusion_matrix(certeza,testY))
        if(i==0):
          CCR_nm[view_g,view_p] = np.round(score*100,2)
        if(i==1):
          CCR_bg[view_g,view_p] = np.round(score*100,2)
        if(i==2):
          CCR_cl[view_g,view_p] = np.round(score*100,2)

      print('Prove view:', prove_view, '\tnm: ',CCR_nm[view_g,view_p],'\tbg: ',CCR_bg[view_g,view_p],'\tcl: ',CCR_cl[view_g,view_p])


  print(str(iteration)+'T Test:1-62',np.round(CCR_nm.mean(),2),'-',np.round(CCR_bg.mean(),2),'-',np.round(CCR_cl.mean(),2))
  print(str(iteration)+'T Test:63-124',np.round(CCR_nm.mean(),2),'-',np.round(CCR_bg.mean(),2),'-',np.round

In [ ]:
file_name='2'
transform_data(verbose=True)
split_flat(verbose=False)
test_model('30')

In [ ]:
import pandas as pd
print('\t    | NORMAL WALKING | ROWS = GALLERY | COLUMNS = PROVE |')
pd.DataFrame(data=CCR_nm, columns=views, index=views)

In [ ]:
print('\t    | CARRYING WALKING | ROWS = GALLERY | COLUMNS = PROVE |')
pd.DataFrame(data=CCR_bg, columns=views, index=views)

In [ ]:
print('\t    | CLOTHING WALKING | ROWS = GALLERY | COLUMNS = PROVE |')
pd.DataFrame(data=CCR_cl, columns=views, index=views)